# FAISS DB 생성

In [16]:
import os
import glob
import re
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# PDF 및 TXT 폴더 경로
pdf_directory = "./pdf"
txt_directory = "./txt"

documents = []

# PDF 파일 로드
pdf_files = glob.glob(os.path.join(pdf_directory, "*.pdf"))
print(f"{len(pdf_files)}개의 PDF 파일을 발견했습니다.")
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    documents.extend(loader.load())

# TXT 파일 로드
txt_files = glob.glob(os.path.join(txt_directory, "*.txt"))
print(f"{len(txt_files)}개의 TXT 파일을 발견했습니다.")
for txt_file in txt_files:
    loader = TextLoader(txt_file, encoding='utf-8')
    documents.extend(loader.load())

print(f"총 {len(documents)}개의 문서 로드 완료")

# 텍스트 전처리 함수
def clean_text(text: str) -> str:
    text = text.replace('\xa0', ' ')
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s.,!?()\[\]{}가-힣A-Za-z0-9]', '', text)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text) # 중복 문자 정규화
    text = re.sub(r'\([^)]*\)', '', text) # 괄호 및 내용 제거
    return text.strip()

# 문서 전처리 적용
for i, doc in enumerate(documents):
    documents[i].page_content = clean_text(doc.page_content)

# 텍스트 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
)

chunks = text_splitter.split_documents(documents)

chunks = [chunk for chunk in chunks if len(chunk.page_content) > 10]  # 빈 청크 제거

print(f"총 {len(chunks)}개의 청크로 분할 완료")

# 임베딩 모델 설정
embeddings = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# faiss 벡터 저장소 생성 및 저장
vector_store = FAISS.from_documents(chunks, embeddings)

# 벡터 저장소 저장 (나중에 재사용할 수 있도록)
vector_store.save_local("faiss_index") 
print("벡터 저장소가 'faiss_index' 폴더에 저장되었습니다.")

31개의 PDF 파일을 발견했습니다.
18개의 TXT 파일을 발견했습니다.
총 227개의 문서 로드 완료
총 1027개의 청크로 분할 완료
벡터 저장소가 'faiss_index' 폴더에 저장되었습니다.


In [18]:
for i, chunk in enumerate(chunks):
    if len(chunk.page_content) < 50:
        print(f"청크 {i} {len(chunk.page_content)}: {chunk.page_content}")
    # print(f"청크 {i} {len(chunk.page_content)}: {chunk.page_content}...")  # 청크의 처음 50자만 출력

청크 183 31: 2025 여행톡톡. All rights reserved.


In [9]:
# 백터 예시
query = '우리나라와 다른 나라의 문화차이'
retrieval = vector_store.similarity_search_with_score(query)
print(retrieval)

[(Document(id='0c01d7e8-ea70-4eac-959a-bdcd95dd4431', metadata={'source': './txt/한국과는 달라도 너무 다른, 미국 LA [문화 전반].txt'}, page_content='문화 차이가 있고, 지역별로 또 다른 문화차이가 존재한다. 이 글은 2년 동안 미국에 살면서 몸소 겪은, 개인적으로 생각했을 때 독특하다고 느꼈던 문화들을 적어 내려간 것이기 때문에 개인적인 견해와 해석이 녹아 있다. 분명한 것은 미국에만 있는 문화인지는 알 수 없지만, 한국에서 나에게 익숙했던 것들이 미국에서는 다른 형태로 존재하기 때문에 간혹 나를 당황스럽게도 만들었고 놀라게도 만들었다는 것이다. 나는 이 글이 미국을 경험해 보지 못한 많은 사람들에게 미국 LA를 조금이나마 선행적으로 이해하고 알 수 있는 간접적인 경험이 되길'), 0.80407023), (Document(id='80a73509-b350-4c4e-8d54-54ffeee2739e', metadata={'source': './txt/한국과는 달라도 너무 다른, 미국 LA [문화 전반].txt'}, page_content='나는 2년 전, 한국에서 미국 LA 땅을 처음 밟았다. 나는 그 이후 LA의 한인 타운에서 계속 살았다. 한인타운은 한국의 문화를 많이 가지고 있어서, 다른 지역에서 살고 있는 한인에 비해 문화차이로 인한 어려움을 많이 겪지 않은 편이다. 그럼에도 불구하고 분명히 한국과는 다른 미국만의 문화가 있다. 미국의 독특한 문화는 아닐지라도, 한국에서는 경험하지 못한 색다른 것들이 때로는 나를 당황스럽게도, 때로는 놀랍게도 만들었다. 아마 미국에 오지 않았다면 경험하지 못했을 몇 가지의 이색적인 미국의 문화들을 여러분들과 나누고 싶다. 1.'), 0.8285365), (Document(id='84462a77-a846-49a7-b292-3ba77123ebab', metadata={'producer': 'Skia/PDF m134', 'creator': 'M

# FAISS DB 불러오기

In [10]:
# 저장된 FAISS 벡터 저장소 불러오기
vector_store = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
print("FAISS 벡터 저장소가 성공적으로 로드되었습니다.")

# 백터 예시
query = '우리나라와 다른 나라의 문화차이'
retrieval = vector_store.similarity_search_with_score(query)
print(retrieval)

FAISS 벡터 저장소가 성공적으로 로드되었습니다.
[(Document(id='0c01d7e8-ea70-4eac-959a-bdcd95dd4431', metadata={'source': './txt/한국과는 달라도 너무 다른, 미국 LA [문화 전반].txt'}, page_content='문화 차이가 있고, 지역별로 또 다른 문화차이가 존재한다. 이 글은 2년 동안 미국에 살면서 몸소 겪은, 개인적으로 생각했을 때 독특하다고 느꼈던 문화들을 적어 내려간 것이기 때문에 개인적인 견해와 해석이 녹아 있다. 분명한 것은 미국에만 있는 문화인지는 알 수 없지만, 한국에서 나에게 익숙했던 것들이 미국에서는 다른 형태로 존재하기 때문에 간혹 나를 당황스럽게도 만들었고 놀라게도 만들었다는 것이다. 나는 이 글이 미국을 경험해 보지 못한 많은 사람들에게 미국 LA를 조금이나마 선행적으로 이해하고 알 수 있는 간접적인 경험이 되길'), 0.80407023), (Document(id='80a73509-b350-4c4e-8d54-54ffeee2739e', metadata={'source': './txt/한국과는 달라도 너무 다른, 미국 LA [문화 전반].txt'}, page_content='나는 2년 전, 한국에서 미국 LA 땅을 처음 밟았다. 나는 그 이후 LA의 한인 타운에서 계속 살았다. 한인타운은 한국의 문화를 많이 가지고 있어서, 다른 지역에서 살고 있는 한인에 비해 문화차이로 인한 어려움을 많이 겪지 않은 편이다. 그럼에도 불구하고 분명히 한국과는 다른 미국만의 문화가 있다. 미국의 독특한 문화는 아닐지라도, 한국에서는 경험하지 못한 색다른 것들이 때로는 나를 당황스럽게도, 때로는 놀랍게도 만들었다. 아마 미국에 오지 않았다면 경험하지 못했을 몇 가지의 이색적인 미국의 문화들을 여러분들과 나누고 싶다. 1.'), 0.8285365), (Document(id='84462a77-a846-49a7-b292-3ba77123ebab', metadata={'producer': '